# Preprocessing the Data

## Import Required Packages

In [2]:
import pandas as pd
import re

pd.set_option("display.max_columns", None)


## Encoding of the Data

NOTE: All the tables are encoded in `UTF-16` format. Python by default assumes `UTF-8` when reading and writing files. Therefore the tables need to either be rewritten in `UTF-8` or the `UTF-16` encoding needs to be specified on read.

## Preprocessing the Fact Table

The fact table `Description` column is not enclosed in quotation marks. Therefore any entries with commas causes parsing errors on read. Luckily, only one text pattern causes this error - commas inside parentheses e.g. a developer joke mug with the quote "(hip, hip, array)". Therefore, this issue can easily be fixed with a find and replace using "simple" regular expressions to remove the erroneous commas with in parentheses.

A more generalizable solution would be to find a way to capture the description text using regular expressions and enclose them in quotation marks. However, the correct regular expression formula will likely be very complex and take time.

#### Reading in the Raw Data

Many rows are skipped on read due to parsing errors.

In [3]:
fact_table = pd.read_csv("data/raw/fact.sale.csv", encoding="utf-16", on_bad_lines="warn")

display(
    fact_table.info(),
    fact_table.head(),
)

Skipping line 9: expected 21 fields, saw 23
Skipping line 12: expected 21 fields, saw 23
Skipping line 19: expected 21 fields, saw 23
Skipping line 51: expected 21 fields, saw 23
Skipping line 52: expected 21 fields, saw 23
Skipping line 68: expected 21 fields, saw 23
Skipping line 71: expected 21 fields, saw 23
Skipping line 104: expected 21 fields, saw 23
Skipping line 109: expected 21 fields, saw 23
Skipping line 185: expected 21 fields, saw 23
Skipping line 270: expected 21 fields, saw 23
Skipping line 277: expected 21 fields, saw 23
Skipping line 283: expected 21 fields, saw 23
Skipping line 289: expected 21 fields, saw 23
Skipping line 290: expected 21 fields, saw 23
Skipping line 314: expected 21 fields, saw 23
Skipping line 342: expected 21 fields, saw 23
Skipping line 366: expected 21 fields, saw 23
Skipping line 410: expected 21 fields, saw 23
Skipping line 441: expected 21 fields, saw 23
Skipping line 493: expected 21 fields, saw 23
Skipping line 538: expected 21 fields, saw

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226121 entries, 0 to 226120
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Sale Key              226121 non-null  int64  
 1   City Key              226121 non-null  int64  
 2   Customer Key          226121 non-null  int64  
 3   Bill To Customer Key  226121 non-null  int64  
 4   Stock Item Key        226121 non-null  int64  
 5   Invoice Date Key      226121 non-null  object 
 6   Delivery Date Key     225839 non-null  object 
 7   Salesperson Key       226121 non-null  int64  
 8   WWI Invoice ID        226121 non-null  int64  
 9   Description           226121 non-null  object 
 10  Package               226121 non-null  object 
 11  Quantity              226121 non-null  int64  
 12  Unit Price            226121 non-null  float64
 13  Tax Rate              226121 non-null  float64
 14  Total Excluding Tax   226121 non-null  float64
 15  

None

,Sale Key,City Key,Customer Key,Bill To Customer Key,Stock Item Key,Invoice Date Key,Delivery Date Key,Salesperson Key,WWI Invoice ID,Description,Package,Quantity,Unit Price,Tax Rate,Total Excluding Tax,Tax Amount,Profit,Total Including Tax,Total Dry Items,Total Chiller Items,Lineage Key
0,49258,71135,0,0,194,2013-10-22,2013-10-23,86,15187,DBA joke mug - SELECT caffeine FROM mug (White),Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11
1,49265,41568,0,0,204,2013-10-22,2013-10-23,83,15189,DBA joke mug - mind if I join you? (White),Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11
2,49456,70409,0,0,202,2013-10-22,2013-10-23,74,15241,DBA joke mug - daaaaaa-ta (White),Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11
3,49372,48937,0,0,173,2013-10-22,2013-10-23,83,15218,Developer joke mug - a foo walks into a bar (B...,Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11
4,49624,41981,0,0,168,2013-10-24,2013-10-25,85,15296,IT joke mug - keyboard not found … press F1 to...,Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11


#### Replacing Erroneous Commas

Since the error is caused by only one repeating pattern, we can fix it with a regular expression to remove all commas within parentheses.

In [4]:
def preprocess_fact_table(filename):
    """Cleans the fact table by removing erroneous commas inside parentheses from inside the CSV file.

    Args:
        filename (str): Name of the raw file.
    """
    
    replace = lambda g: g.group(0).replace(",", "")
    
    # Read in the file.
    with open(f"data/raw/{filename}", "r", encoding="utf-16") as file :
        filedata = file.read()

    # Remove commas inside parentheses.
    filedata_preprocessed = re.sub(r"\(.*?\)", replace, filedata)

    # Write the preprocessed file out again in UTF-8.
    with open(f"data/preprocessed/{filename}", "w") as file:
        file.write(filedata_preprocessed)
        

preprocess_fact_table("fact.sale.csv")

#### Reading in the Preprocessed Table

Note the encoding flag is no longer required, as the preprocess data is now in the default encoding used by Python.

In [5]:
fact_table = pd.read_csv("data/preprocessed/fact.sale.csv")

display(
    fact_table.info(),
    fact_table.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228265 entries, 0 to 228264
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Sale Key              228265 non-null  int64  
 1   City Key              228265 non-null  int64  
 2   Customer Key          228265 non-null  int64  
 3   Bill To Customer Key  228265 non-null  int64  
 4   Stock Item Key        228265 non-null  int64  
 5   Invoice Date Key      228265 non-null  object 
 6   Delivery Date Key     227981 non-null  object 
 7   Salesperson Key       228265 non-null  int64  
 8   WWI Invoice ID        228265 non-null  int64  
 9   Description           228265 non-null  object 
 10  Package               228265 non-null  object 
 11  Quantity              228265 non-null  int64  
 12  Unit Price            228265 non-null  float64
 13  Tax Rate              228265 non-null  float64
 14  Total Excluding Tax   228265 non-null  float64
 15  

None

,Sale Key,City Key,Customer Key,Bill To Customer Key,Stock Item Key,Invoice Date Key,Delivery Date Key,Salesperson Key,WWI Invoice ID,Description,Package,Quantity,Unit Price,Tax Rate,Total Excluding Tax,Tax Amount,Profit,Total Including Tax,Total Dry Items,Total Chiller Items,Lineage Key
0,49258,71135,0,0,194,2013-10-22,2013-10-23,86,15187,DBA joke mug - SELECT caffeine FROM mug (White),Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11
1,49265,41568,0,0,204,2013-10-22,2013-10-23,83,15189,DBA joke mug - mind if I join you? (White),Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11
2,49456,70409,0,0,202,2013-10-22,2013-10-23,74,15241,DBA joke mug - daaaaaa-ta (White),Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11
3,49372,48937,0,0,173,2013-10-22,2013-10-23,83,15218,Developer joke mug - a foo walks into a bar (B...,Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11
4,49624,41981,0,0,168,2013-10-24,2013-10-25,85,15296,IT joke mug - keyboard not found … press F1 to...,Each,6,13.0,15.0,78.0,11.7,51.0,89.7,6,0,11


## Preprocessing the Customer Dimension Table

### Reading in the Raw Data

Once again many rows are skipped on read due to parsing errors.

In [6]:
customer_dim = pd.read_csv("data/raw/dim.customer.csv", encoding="utf-16", on_bad_lines="warn")

display(
    customer_dim.info(),
    customer_dim.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Customer Key      8 non-null      object 
 1   WWI Customer ID   3 non-null      float64
 2   Customer          3 non-null      object 
 3   Bill To Customer  2 non-null      object 
 4   Category          2 non-null      object 
 5   Buying Group      2 non-null      object 
 6   Primary Contact   2 non-null      object 
 7   Postal Code       2 non-null      float64
 8   Valid From        3 non-null      object 
 9   Valid To          3 non-null      object 
 10  Lineage Key       3 non-null      float64
dtypes: float64(3), object(8)
memory usage: 832.0+ bytes


Skipping line 4: expected 11 fields, saw 12
Skipping line 5: expected 11 fields, saw 12
Skipping line 6: expected 11 fields, saw 12
Skipping line 7: expected 11 fields, saw 12
Skipping line 8: expected 11 fields, saw 12
Skipping line 9: expected 11 fields, saw 12
Skipping line 10: expected 11 fields, saw 12
Skipping line 11: expected 11 fields, saw 12
Skipping line 12: expected 11 fields, saw 12
Skipping line 13: expected 11 fields, saw 12
Skipping line 14: expected 11 fields, saw 12
Skipping line 15: expected 11 fields, saw 12
Skipping line 16: expected 11 fields, saw 12
Skipping line 17: expected 11 fields, saw 12
Skipping line 18: expected 11 fields, saw 12
Skipping line 19: expected 11 fields, saw 12
Skipping line 20: expected 11 fields, saw 12
Skipping line 21: expected 11 fields, saw 12
Skipping line 22: expected 11 fields, saw 12
Skipping line 23: expected 11 fields, saw 12
Skipping line 24: expected 11 fields, saw 12
Skipping line 25: expected 11 fields, saw 12
Skipping line 26

None

,Customer Key,WWI Customer ID,Customer,Bill To Customer,Category,Buying Group,Primary Contact,Postal Code,Valid From,Valid To,Lineage Key
0,0,0.0,Unknown,NaN,NaN,NaN,NaN,NaN,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,0.0
1,1,1.0,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410.0,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,2.0
2,70|70|Tailspin Toys (New Baden| IL)|Tailspin T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71|71|Tailspin Toys (Good Hart| MI)|Tailspin T...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,72|72|Tailspin Toys (Cortaro| AZ)|Tailspin Toy...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Replacing Pipes With Commas and Removing Erroneous Commas
Some rows have accidentally replaced commas with a pipe symbol (`|`). Like before, the parsing error is due to the same text pattern as before (extra commas inside parentheses).

In [7]:
def preprocess_customer_table(filename):
    """Cleans customer table by replacing commas with pipes then removing erroneous commas inside parentheses from the CSV file.

    Args:
        filename (str): Name of the raw file.
    """
    
    replace = lambda g: g.group(0).replace(",", "")
    
    # Read in the file.
    with open(f"data/raw/{filename}", "r", encoding="utf-16") as file :
        filedata = file.read()
    
    # Replace pipes with commas.
    replace_pipes = filedata.replace("|", ",")

    # Remove commas inside parentheses
    filedata_preprocessed = re.sub(r"\(.*?\)", replace, replace_pipes)

    # Write the preprocessed file out again in UTF-8.
    with open(f"data/preprocessed/{filename}", "w") as file:
        file.write(filedata_preprocessed)
        
preprocess_customer_table("dim.customer.csv")

#### Reading in the Preprocessed Data

In [8]:
customer_dim = pd.read_csv("data/preprocessed/dim.customer.csv")

display(
    customer_dim.info(),
    customer_dim.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Customer Key      403 non-null    int64  
 1   WWI Customer ID   403 non-null    int64  
 2   Customer          403 non-null    object 
 3   Bill To Customer  402 non-null    object 
 4   Category          402 non-null    object 
 5   Buying Group      402 non-null    object 
 6   Primary Contact   402 non-null    object 
 7   Postal Code       402 non-null    float64
 8   Valid From        403 non-null    object 
 9   Valid To          403 non-null    object 
 10  Lineage Key       403 non-null    int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 34.8+ KB


None

,Customer Key,WWI Customer ID,Customer,Bill To Customer,Category,Buying Group,Primary Contact,Postal Code,Valid From,Valid To,Lineage Key
0,0,0,Unknown,NaN,NaN,NaN,NaN,NaN,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,0
1,1,1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410.0,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,2
2,2,2,Tailspin Toys (Sylvanite MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216.0,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,2
3,3,3,Tailspin Toys (Peeples Valley AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205.0,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,2
4,4,4,Tailspin Toys (Medicine Lodge KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152.0,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,2


## Preprocessing Date Dimension Table

This table already looks fairly clean on inspection and is not returning any parsing errors. All that's likely required is to change the encoding to `UTF-8` for ease of reading in Python.

In [9]:
date_dim = pd.read_csv("data/raw/dim.date.csv", encoding="utf-16", on_bad_lines="warn")

display(
    date_dim.info(),
    date_dim.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461 entries, 0 to 1460
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Date                   1461 non-null   object
 1   Day Number             1461 non-null   int64 
 2   Day                    1461 non-null   int64 
 3   Month                  1461 non-null   object
 4   Short Month            1461 non-null   object
 5   Calendar Month Number  1461 non-null   int64 
 6   Calendar Month Label   1461 non-null   object
 7   Calendar Year          1461 non-null   int64 
 8   Calendar Year Label    1461 non-null   object
 9   Fiscal Month Number    1461 non-null   int64 
 10  Fiscal Month Label     1461 non-null   object
 11  Fiscal Year            1461 non-null   int64 
 12  Fiscal Year Label      1461 non-null   object
 13  ISO Week Number        1461 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 159.9+ KB


None

,Date,Day Number,Day,Month,Short Month,Calendar Month Number,Calendar Month Label,Calendar Year,Calendar Year Label,Fiscal Month Number,Fiscal Month Label,Fiscal Year,Fiscal Year Label,ISO Week Number
0,2013-01-01,1,1,January,Jan,1,CY2013-Jan,2013,CY2013,3,FY2013-Jan,2013,FY2013,1
1,2013-01-02,2,2,January,Jan,1,CY2013-Jan,2013,CY2013,3,FY2013-Jan,2013,FY2013,1
2,2013-01-03,3,3,January,Jan,1,CY2013-Jan,2013,CY2013,3,FY2013-Jan,2013,FY2013,1
3,2013-01-04,4,4,January,Jan,1,CY2013-Jan,2013,CY2013,3,FY2013-Jan,2013,FY2013,1
4,2013-01-05,5,5,January,Jan,1,CY2013-Jan,2013,CY2013,3,FY2013-Jan,2013,FY2013,1


### Fix Encoding

In [10]:
def fix_encoding(filename):
    """Rewrite file in UTF-8 encoding.

    Args:
        filename (str): Name of the raw file.
    """
    # Read in the file.
    with open(f"data/raw/{filename}", "r", encoding="utf-16") as file :
        filedata = file.read()

    # Write the file out again in UTF-8.
    with open(f"data/preprocessed/{filename}", "w") as file:
        file.write(filedata)
        
fix_encoding("dim.date.csv")

## Preprocessing Employee Dimension Table

Some rows are skipped due to parsing errors. This is because one employee "Ethan Onslow" was occasionally inputted as "Ethan, Onslow". This can be fixed with a simple hardcoded find and replace. A generalizable solution is preferable, however for now this approach is acceptable.

In [11]:
employee_dim = pd.read_csv("data/raw/dim.employee.csv", encoding="utf-16", on_bad_lines="warn")

display(
    employee_dim.info(),
    employee_dim.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Employee Key     206 non-null    int64 
 1   WWI Employee ID  206 non-null    int64 
 2   Employee         206 non-null    object
 3   Preferred Name   205 non-null    object
 4   Is Salesperson   206 non-null    bool  
 5   Photo            206 non-null    object
 6   Valid From       206 non-null    object
 7   Valid To         206 non-null    int64 
 8   Lineage Key      206 non-null    int64 
dtypes: bool(1), int64(4), object(4)
memory usage: 13.2+ KB


Skipping line 69: expected 9 fields, saw 10
Skipping line 81: expected 9 fields, saw 10
Skipping line 89: expected 9 fields, saw 10
Skipping line 150: expected 9 fields, saw 10
Skipping line 155: expected 9 fields, saw 10
Skipping line 162: expected 9 fields, saw 10
Skipping line 168: expected 9 fields, saw 10



None

,Employee Key,WWI Employee ID,Employee,Preferred Name,Is Salesperson,Photo,Valid From,Valid To,Lineage Key
0,0,0,Unknown,NaN,False,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,0,0
1,1,14,Lily Code,Lily,True,2013-01-01 00:00:00.0000000,2013-01-01 08:00:00.0000000,3,3
2,2,4,Isabella Rupp,Isabella,False,2013-01-01 00:00:00.0000000,2013-01-01 08:00:00.0000000,3,3
3,3,11,Ethan Onslow,Ethan,False,2013-01-01 00:00:00.0000000,2013-01-04 08:00:00.0000000,3,3
4,4,7,Amy Trefl,Amy,True,2013-01-01 00:00:00.0000000,2013-01-05 08:00:00.0000000,3,3


### Remove Erroneous Commas

In [14]:
def preprocess_employee_table(filename):
    """Cleans employee table by removing erroneous commas from the CSV file.

    Args:
        filename (str): Name of the raw file.
    """
    
    # Read in the file.
    with open(f"data/raw/{filename}", "r", encoding="utf-16") as file :
        filedata = file.read()
    
    # Replace erroneous employee name. TODO: Generalize without hard coding.
    filedata_preprocessed = filedata.replace("Ethan, Onslow", "Ethan Onslow")

    # Write the preprocessed file out again in UTF-8.
    with open(f"data/preprocessed/{filename}", "w") as file:
        file.write(filedata_preprocessed)
        
preprocess_employee_table("dim.employee.csv")

### Read in Preprocessed Data

In [17]:
employee_dim = pd.read_csv("data/preprocessed/dim.employee.csv")

display(
    employee_dim.info(),
    employee_dim.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Employee Key     213 non-null    int64 
 1   WWI Employee ID  213 non-null    int64 
 2   Employee         213 non-null    object
 3   Preferred Name   212 non-null    object
 4   Is Salesperson   213 non-null    bool  
 5   Photo            213 non-null    object
 6   Valid From       213 non-null    object
 7   Valid To         213 non-null    int64 
 8   Lineage Key      213 non-null    int64 
dtypes: bool(1), int64(4), object(4)
memory usage: 13.6+ KB


None

,Employee Key,WWI Employee ID,Employee,Preferred Name,Is Salesperson,Photo,Valid From,Valid To,Lineage Key
0,0,0,Unknown,NaN,False,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,0,0
1,1,14,Lily Code,Lily,True,2013-01-01 00:00:00.0000000,2013-01-01 08:00:00.0000000,3,3
2,2,4,Isabella Rupp,Isabella,False,2013-01-01 00:00:00.0000000,2013-01-01 08:00:00.0000000,3,3
3,3,11,Ethan Onslow,Ethan,False,2013-01-01 00:00:00.0000000,2013-01-04 08:00:00.0000000,3,3
4,4,7,Amy Trefl,Amy,True,2013-01-01 00:00:00.0000000,2013-01-05 08:00:00.0000000,3,3


## Preprocessing the Payment Dimension Table

On inspection is table looks fine. Only the encoding needs to be fixed.

In [12]:
payment_dim = pd.read_csv("data/raw/dim.payment.csv", encoding="utf-16", on_bad_lines="warn")

display(
    payment_dim.info(),
    payment_dim.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Payment Method Key     6 non-null      int64 
 1   WWI Payment Method ID  6 non-null      int64 
 2   Payment Method         6 non-null      object
 3   Valid From             6 non-null      object
 4   Valid To               6 non-null      object
 5   Lineage Key            6 non-null      int64 
dtypes: int64(3), object(3)
memory usage: 416.0+ bytes


None

,Payment Method Key,WWI Payment Method ID,Payment Method,Valid From,Valid To,Lineage Key
0,0,0,Unknown,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,0
1,1,3,Credit Card,2013-01-01 00:00:00.0000000,2016-01-01 16:00:00.0000000,4
2,2,1,Cash,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,4
3,3,2,Check,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,4
4,4,4,EFT,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,4


In [15]:
fix_encoding("dim.payment.csv")

## Preprocessing Stock Item Dimension Table

Again suffers from parsing errors due to extra commas in parentheses. 

In [18]:
stockitem_dim = pd.read_csv("data/raw/dim.stockItem.csv", encoding="utf-16", on_bad_lines="warn")

display(
    stockitem_dim.info(),
    stockitem_dim.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666 entries, 0 to 665
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Stock Item Key            666 non-null    int64  
 1   WWI Stock Item ID         666 non-null    int64  
 2   Stock Item                666 non-null    object 
 3   Color                     354 non-null    object 
 4   Selling Package           665 non-null    object 
 5   Buying Package            665 non-null    object 
 6   Brand                     67 non-null     object 
 7   Size                      468 non-null    object 
 8   Lead Time Days            666 non-null    int64  
 9   Quantity Per Outer        666 non-null    int64  
 10  Is Chiller Stock          666 non-null    bool   
 11  Barcode                   16 non-null     float64
 12  Tax Rate                  666 non-null    float64
 13  Unit Price                666 non-null    float64
 14  Recommende

Skipping line 181: expected 20 fields, saw 22
Skipping line 182: expected 20 fields, saw 22
Skipping line 267: expected 20 fields, saw 22
Skipping line 268: expected 20 fields, saw 22
Skipping line 575: expected 20 fields, saw 22
Skipping line 576: expected 20 fields, saw 22



None

,Stock Item Key,WWI Stock Item ID,Stock Item,Color,Selling Package,Buying Package,Brand,Size,Lead Time Days,Quantity Per Outer,Is Chiller Stock,Barcode,Tax Rate,Unit Price,Recommended Retail Price,Typical Weight Per Unit,Photo,Valid From,Valid To,Lineage Key
0,0,0,Unknown,NaN,NaN,NaN,NaN,NaN,0,0,False,NaN,0.0,0.0,0.00,0.00,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,0,0
1,1,219,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,NaN,14.0,50.0,74.75,1.00,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,5
2,2,218,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,NaN,14.0,37.5,56.06,0.75,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,5
3,3,217,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,NaN,14.0,25.0,37.38,0.50,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,5
4,4,216,Void fill 100 L bag (White) 100L,NaN,Each,Each,NaN,100L,14,10,False,NaN,14.0,12.5,18.69,0.25,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,5


### Removing Erroneous Commas

In [19]:
def preprocess_stockitem_table(filename):
    """Cleans the stockitem table by removing erroneous commas inside parentheses from the CSV file.

    Args:
        filename (str): Name of the raw file.
    """
    
    replace = lambda g: g.group(0).replace(",", "")
    
    # Read in the file.
    with open(f"data/raw/{filename}", "r", encoding="utf-16") as file :
        filedata = file.read()

    # Remove commas inside parentheses.
    filedata_preprocessed = re.sub(r"\(.*?\)", replace, filedata)

    # Write the preprocessed file out again in UTF-8.
    with open(f"data/preprocessed/{filename}", "w") as file:
        file.write(filedata_preprocessed)
        
preprocess_stockitem_table("dim.stockItem.csv")

### Read in Preprocessed Data

In [20]:
stockitem_dim = pd.read_csv("data/preprocessed/dim.stockItem.csv")

display(
    stockitem_dim.info(),
    stockitem_dim.head(),
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Stock Item Key            672 non-null    int64  
 1   WWI Stock Item ID         672 non-null    int64  
 2   Stock Item                672 non-null    object 
 3   Color                     360 non-null    object 
 4   Selling Package           671 non-null    object 
 5   Buying Package            671 non-null    object 
 6   Brand                     67 non-null     object 
 7   Size                      468 non-null    object 
 8   Lead Time Days            672 non-null    int64  
 9   Quantity Per Outer        672 non-null    int64  
 10  Is Chiller Stock          672 non-null    bool   
 11  Barcode                   16 non-null     float64
 12  Tax Rate                  672 non-null    float64
 13  Unit Price                672 non-null    float64
 14  Recommende

None

,Stock Item Key,WWI Stock Item ID,Stock Item,Color,Selling Package,Buying Package,Brand,Size,Lead Time Days,Quantity Per Outer,Is Chiller Stock,Barcode,Tax Rate,Unit Price,Recommended Retail Price,Typical Weight Per Unit,Photo,Valid From,Valid To,Lineage Key
0,0,0,Unknown,NaN,NaN,NaN,NaN,NaN,0,0,False,NaN,0.0,0.0,0.00,0.00,2013-01-01 00:00:00.0000000,9999-12-31 23:59:59.9999999,0,0
1,1,219,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,NaN,14.0,50.0,74.75,1.00,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,5
2,2,218,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,NaN,14.0,37.5,56.06,0.75,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,5
3,3,217,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,NaN,14.0,25.0,37.38,0.50,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,5
4,4,216,Void fill 100 L bag (White) 100L,NaN,Each,Each,NaN,100L,14,10,False,NaN,14.0,12.5,18.69,0.25,2013-01-01 00:00:00.0000000,2016-05-31 23:00:00.0000000,5,5
